In [2]:
import numpy as np
import matplotlib.pyplot as plt 
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras import regularizers
import h5py
import sys
%matplotlib inline 


ModuleNotFoundError: No module named 'tensorflow'

In [1]:
import tensorflow as tf
with tf.device('/gpu:0'):
    a = tf.constant([1.0, 2.0, 3.0, 4.0, 5.0, 6.0], shape=[2, 3], name='a')
    b = tf.constant([1.0, 2.0, 3.0, 4.0, 5.0, 6.0], shape=[3, 2], name='b')
    c = tf.matmul(a, b)

with tf.Session() as sess:
    print (sess.run(c))

ModuleNotFoundError: No module named 'tensorflow'

In [2]:
be = h5py.File('variables_tt_re.h5','r')
bmu = h5py.File('variables_tt_rmu.h5','r')
be1 = h5py.File('variables_tt_re1.h5','r')
print(be.keys())
print('')
print(be1.keys())
dataset = be1

<KeysViewHDF5 ['j1_eta', 'j1_isbtag', 'j1_m', 'j1_phi', 'j1_pt', 'j2_eta', 'j2_isbtag', 'j2_m', 'j2_phi', 'j2_pt', 'j3_eta', 'j3_isbtag', 'j3_m', 'j3_phi', 'j3_pt', 'j4_eta', 'j4_isbtag', 'j4_m', 'j4_phi', 'j4_pt', 'j5_eta', 'j5_isbtag', 'j5_m', 'j5_phi', 'j5_pt', 'j6_eta', 'j6_isbtag', 'j6_m', 'j6_phi', 'j6_pt', 'j7_eta', 'j7_isbtag', 'j7_m', 'j7_phi', 'j7_pt', 'j8_eta', 'j8_isbtag', 'j8_m', 'j8_phi', 'j8_pt', 'lep_eta', 'lep_m', 'lep_phi', 'lep_pt', 'met_met', 'met_phi', 'mttReco', 'mttTrue']>

<KeysViewHDF5 ['bh_eta', 'bh_phi', 'bh_pt', 'bl_eta', 'bl_phi', 'bl_pt', 'j1_DL1r', 'j1_eta', 'j1_isbtag', 'j1_m', 'j1_phi', 'j1_pt', 'j2_DL1r', 'j2_eta', 'j2_isbtag', 'j2_m', 'j2_phi', 'j2_pt', 'j3_DL1r', 'j3_eta', 'j3_isbtag', 'j3_m', 'j3_phi', 'j3_pt', 'j4_DL1r', 'j4_eta', 'j4_isbtag', 'j4_m', 'j4_phi', 'j4_pt', 'j5_DL1r', 'j5_eta', 'j5_isbtag', 'j5_m', 'j5_phi', 'j5_pt', 'j6_DL1r', 'j6_eta', 'j6_isbtag', 'j6_m', 'j6_phi', 'j6_pt', 'j7_DL1r', 'j7_eta', 'j7_isbtag', 'j7_m', 'j7_phi', 'j7_pt'

In [3]:
input_keys = ['j1_pt', 'j1_eta', 'j1_phi', 'j1_m', 'j1_DL1r', 'j2_pt', 'j2_eta', 'j2_phi', 'j2_m', 'j2_DL1r', 'j3_pt', 'j3_eta', 'j3_phi', 'j3_m', 'j3_DL1r', 'j4_pt', 'j4_eta', 'j4_phi', 'j4_m', 'j4_DL1r', 'j5_pt', 'j5_eta', 'j5_phi', 'j5_m', 'j5_DL1r', 'j6_pt', 'j6_eta', 'j6_phi', 'j6_m', 'j6_DL1r', 'j7_pt', 'j7_eta', 'j7_phi','j7_m', 'j7_DL1r', 'j8_pt', 'j8_eta', 'j8_phi','j8_m', 'j8_DL1r', 'lep_pt', 'lep_eta', 'lep_phi', 'lep_m', 'met_met', 'met_phi'] 
output_keys = ['th_pt']

In [4]:
# Convert into numpy arrays and mean normalize
crop = 150000 # len(dataset.get('j1_pt'))

input_arrays = []
output_arrays = [] 

for key in input_keys:
    exec(key + ' = np.array(dataset.get(key))[0:crop].reshape((1,-1))')
    exec('{0} = ({0} - np.mean({0}))/np.std({0})'.format(key))
    exec('input_arrays.append({0})'.format(key))

for key in output_keys:
    exec(key + ' = np.array(dataset.get(key))[0:crop].reshape((1,-1))')
    exec('{0} = ({0} - np.mean({0}))/np.std({0})'.format(key))
    exec('output_arrays.append({0})'.format(key))

In [5]:
# Create input and test array
split = int(np.floor(0.9*crop)) # 80/20 split 
shuffle = True
    
total_input = np.stack(input_arrays).reshape((crop, len(input_keys)))
total_output = np.stack(output_arrays).reshape((crop, len(output_keys)))

if shuffle:
    rng_state = np.random.get_state()
    np.random.shuffle(total_input)
    np.random.set_state(rng_state)
    np.random.shuffle(total_output)
    
# size = total_input.shape
# total_input = np.array(np.random.normal(size=total_input.shape))

train_input, test_input = total_input[0:split, :], total_input[split:, :]
train_output, test_output = total_output[0:split,:], total_output[split:,:]

In [6]:
print(train_input.shape, train_output.shape)
print(test_input.shape, test_output.shape)

(135000, 46) (135000, 1)
(15000, 46) (15000, 1)


In [7]:
# Build model

optimizer = keras.optimizers.RMSprop(learning_rate = 1e-4)
# optimizer = keras.optimizers.Adam(learning_rate=1e-5, beta_1=0.2)

def build_model():
    model = keras.Sequential([
        layers.Dense(64, activation='tanh', 
                     input_shape=[len(input_keys)], 
                     kernel_regularizer=regularizers.l2(0.00010)), 
        layers.BatchNormalization(),
        layers.Dense(32, activation='linear'),
        layers.Dense(1)
    ])
    
    model.compile(loss='mse', optimizer= optimizer, metrics=['mse'])
    
    return model 
    

In [8]:
model = build_model()
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 64)                3008      
_________________________________________________________________
batch_normalization (BatchNo (None, 64)                256       
_________________________________________________________________
dense_1 (Dense)              (None, 32)                2080      
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 33        
Total params: 5,377
Trainable params: 5,249
Non-trainable params: 128
_________________________________________________________________


In [ ]:
# Fit model

Epochs= 16

history = model.fit(train_input, train_output, verbose=1, epochs=Epochs, 
                   validation_split=0.2, 
                   batch_size=60)


Train on 108000 samples, validate on 27000 samples
Epoch 1/16
108000/108000 [==============================] - 6s 58us/sample - loss: 1.1276 - mse: 1.1223 - val_loss: 1.0267 - val_mse: 1.0214
Epoch 2/16
108000/108000 [==============================] - 5s 47us/sample - loss: 1.0232 - mse: 1.0179 - val_loss: 1.0120 - val_mse: 1.0068
Epoch 3/16
108000/108000 [==============================] - 5s 49us/sample - loss: 1.0130 - mse: 1.0079 - val_loss: 1.0063 - val_mse: 1.0012
Epoch 4/16
108000/108000 [==============================] - 5s 44us/sample - loss: 1.0094 - mse: 1.0044 - val_loss: 1.0052 - val_mse: 1.0003
Epoch 5/16
107880/108000 [============================>.] - ETA: 0s - loss: 1.0085 - mse: 1.0036

In [ ]:
history.history.keys()

In [ ]:
plt.figure(figsize=(12,8))
plt.plot(history.history['loss'], label='training')
plt.plot(history.history['val_loss'], label='validation')
plt.xlabel('epoch')
plt.ylabel('mse loss')
plt.legend()
plt.title('MSE loss')

In [ ]:
predictions = model.predict(test_input)

In [ ]:
predictions_small = predictions[0:100,:]
test_output_small = test_output[0:100,:]

plt.figure(figsize=(4,24))
for i in range(0,1):
    plt.subplot(6,1,i+1)
    plt.plot(predictions_small[:,i],label = 'Predictions')
    plt.plot(test_output_small[:,i],label = 'True Value')
    plt.legend()
    